Data we are interested:
1) snow cover -- sentinel-2
2) land cover type
3) ASO SWE
4) GPR SWE

In [4]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import earthaccess
import xarray as xr
import pprint 
import logging 
logging.basicConfig(level=logging.INFO)
earthaccess.login()

start = datetime(2019, 10, 1)
end = datetime(2020, 5, 1)


In [8]:
query = earthaccess.search_datasets(
    keyword = 'HLS'
)


INFO:earthaccess.api:Datasets found: 15


In [9]:
for collection in query:
    pprint.pprint(collection.summary(), sort_dicts = True, indent=4)

{   'cloud-info': {   'Region': 'us-west-2',
                      'S3BucketAndObjectPrefixNames': [   's3://lp-prod-protected/HLSL30.020',
                                                          's3://lp-prod-public/HLSL30.020'],
                      'S3CredentialsAPIDocumentationURL': 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentialsREADME',
                      'S3CredentialsAPIEndpoint': 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'},
    'concept-id': 'C2021957657-LPCLOUD',
    'file-type': "[{'Format': 'Cloud Optimized GeoTIFF (COG)', 'FormatType': "
                 "'Native', 'Media': ['Earthdata Cloud', 'HTTPS'], "
                 "'AverageFileSize': 20.0, 'AverageFileSizeUnit': 'MB', "
                 "'TotalCollectionFileSizeBeginDate': "
                 "'2013-04-11T00:00:00.000Z'}]",
    'get-data': [   'https://search.earthdata.nasa.gov/search?q=C2021957657-LPCLOUD',
                    'https://appeears.earthdatacloud.nasa.gov/'],
    'short

# Sentinel-2 snow cover 

In [ ]:
# get HLS data from earthdata 
def search_stac_for_HLS(pt, dt_min, dt_max, cloudcover_max=80, lim=100, url='https://cmr.earthdata.nasa.gov/stac/LPCLOUD', collections=['HLSL30.v2.0', 'HLSS30.v2.0']):
    # open the catalog
    catalog = Client.open(f'{url}')
    
    # perform the search
    search = catalog.search(
        collections=collections,
        intersects=pt,
        datetime=dt_min + '/' + dt_max,
        limit=lim
    )

    links = []

    if search.matched() == 0:
        print('No granules found at point', pt, 'from', dt_min, 'to', dt_max)
    else:
        print('Found', search.matched(), 'granules at point', pt, 'from', dt_min, 'to', dt_max)
        item_collection = search.get_all_items()
        
        for i in item_collection:
            if i.properties['eo:cloud_cover'] <= cloudcover_max:
                if len(links) == 0:
                    print(i.properties)
                for a in i.assets:
                    links.append(i.assets[a].href)

    return(links)

def fix_links(src_link, src_dirs, dst_dir, meta_dir, add_tile_dir=True):
    dst_link = src_link

    if '.xml' in dst_link:
        dst_link2 = os.path.join(meta_dir, os.path.basename(dst_link))
    else:
        for src_dir in src_dirs:
            dst_link = dst_link.replace(src_dir, dst_dir)
        dst_splits = dst_link.split('/')
        dst_link2 = '/'.join(dst_splits[0:2]) + \
            '/' + dst_splits[3].split('.')[2] + \
            '/' + '/'.join(dst_splits[3:])
    
    return(dst_link2)

#fix_links(s3_links[0], 's3://lp-prod-protected', './HLS_data')
local_links = [fix_links(src_link=l, src_dirs=['s3://lp-prod-protected', 's3://lp-prod-public'], dst_dir='./HLS_data', meta_dir='./HLS_metadata') for l in s3_links]
print(local_links[0:10])

In [ ]:
%%time

s3 = session.client('s3')

def download_data(s3_links, local_links, s3_session):
    s3_links = [l.replace('s3://', '') for l in s3_links]
    
    for i in range(0, len(s3_links)):
        s3_link = s3_links[i]
        s3_bucket = s3_link.split('/')[0]
        s3_link = s3_link.replace(s3_bucket +'/', '')        
        local_link = local_links[i]
        
        # ignore XML files for now, figure out how to get them later because they contain useful information
        if not '.xml' in local_link:
            with open(local_link, 'wb') as f:
                #print(i, s3_bucket, s3_link, local_link)
                s3.download_fileobj(s3_bucket, s3_link, f)


download_data(s3_links, local_links, s3)


In [ ]:
hls_links = search_stac_for_HLS(pt, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
# convert https links to s3 links
s3_links = [l.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://') for l in hls_links]
s3_links = [l.replace('https://cmr.earthdata.nasa.gov/', 's3://') for l in s3_links]
print(s3_links[0:20])